# Churn Modelling EDA — STARTER NOTEBOOK

**Data Analytics @ Newman University**


## About this Data Set
**This data is from [the Churn-Modelling data set from Kaggle](https://www.kaggle.com/shubh0799/churn-modelling).**<br/>
**Number of Records:** 10,000<br/>
**Number of original fields:** 14 (including a supplied index)<br/>
**Fields include:**
- `RowNumber` - a supplied index
- `CustomerId` - unique ID number for each customer
- `Surname` - customer last name
- `CreditScore` - customer credit score
- `Geography` - the country in which the customer resides
- `Gender` - Male or Female
- `Age` - customer's age as integer
- `Tenure` - number of years as a customer, in integers
- `Balance` - customer's total bank balance
- `NumOfProducts` - the number of banking products a custom participates in
- `HasCrCard` - binary 0 or 1 indicating whether the customer has a bank credit card
- `IsActiveMember` - binary 0 or 1 indicating whether the customer has been active within past ?? time period
- `EstimatedSalary` - the customer's estimated salary
- `Exited` - binary 0 or 1 indicating whether the customer has left the bank and closed all accounts

# Import Libraries & Set Default Plot Attributes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Customize seaborn plot styles
# Seaborn docs: https://seaborn.pydata.org/tutorial/aesthetics.html

# Adjust to retina quality
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Adjust dpi and font size
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook', font_scale = 0.8)

# Display tick marks
sns.set_style('ticks')

# Remove borders
plt.rc('axes.spines', top=False, right=False, left=False, bottom=False)

In [ ]:
# Color palettes for plots
# Named colors: https://matplotlib.org/stable/gallery/color/named_colors.html
# Seaborn color palette docs: https://seaborn.pydata.org/tutorial/color_palettes.html
# Seaborn palette chart: https://www.codecademy.com/article/seaborn-design-ii

# cp1 Color Palette - a binary blue/orange palette
blue = 'deepskyblue' # Use 'skyblue' for a lighter blue
orange = 'orange'
cp1 = [blue, orange]

# cp2 Palette - Reversed binary color order when needed for certain plots
cp2 = [orange, blue]

# cp5 Palette - 5 colors for use with categorical data
turqoise = 'mediumaquamarine'
salmon = 'darksalmon'
tan = 'tan'
gray = 'darkgray'
cp5 = [blue, turqoise, salmon, tan, gray]

# cpd Palette - blue-to-orange diverging palette for correlation heatmaps
cpd = sns.diverging_palette(242, 39, s=100, l=65, n=11)

# Set the default palette
sns.set_palette(cp1)

In [ ]:
# View cp1 color palette
sns.palplot(cp1)

In [ ]:
# View cp2 color palette
sns.palplot(cp2)

In [ ]:
# View cp5 color palette
sns.palplot(cp5)

In [ ]:
# View cpd color palette
sns.palplot(cpd)

# Read and Review the Data

In [ ]:
# If using Google Colab, uncomment the following two lines
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Read the data
# Use the provided RowNumber column as the index
df = pd.read_csv('data/Churn_Modelling.csv', index_col = 'RowNumber')
df.head(10)

In [ ]:
# Dataframe fundamental info
df.info()

**NOTE: _No missing data!_** That's an issue we don't have to worry about with this data set.

In [ ]:
# Check number of unique values per field
df.nunique()

**Notes**:
- There are 10,000 unique CustomerId's so we assume 10,000 unique customers.
- Many customers share last names, but we will assume that's a reasonable proportion.
- Fields with only a handful of unique values can be considered categorical (for text fields like Geography) or ordinal (for fields like NumOfProducts).
- Four fields are binary (yes/no): Gender, HasCrCard, IsActiveMember, Exited (the target variable).

# Drop Irrelevant Columns

**These have no predictive value**:
- CustomerId
- Surname

In [ ]:
# Drop CustomerId and Surname

## Explore Target Variable: Exited
- Exited = 0 --> did not leave our company
- Exited = 1 --> left the company

In [ ]:
# Of these 10,000, how many exited?
df['Exited'].value_counts()

In [ ]:
# Countplot
plt.title("Count of Exited: 0 or 1", fontsize=14, fontweight='bold')
ax = sns.countplot(data=df, x='Exited');
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

**2037 or 20.37% of our customers exited.**

# Explore Continuous Features
We'll explore these features, which are continuous:
- CreditScore
- Age
- Tenure
- Balance
- Estimated Salary

Keep Exited as the target variable for analysis

In [ ]:
# Create a variable to filter for our continuous features
cont = ['Exited','CreditScore','Age','Tenure','Balance','EstimatedSalary']

In [ ]:
# View summary stats for continuous features
# Round the output to two decimals for easier reading. See: https://gist.github.com/data-enhanced/0c48586f36e8cd27ffad8fe076d62c4e
df[cont].describe().apply(lambda s: s.apply('{:,.2f}'.format))

In [ ]:
# Group by Exited and view the mean for each continuous variable
# Round the output to two decimals for easier reading
df[cont].groupby('Exited').mean().apply(lambda s: s.apply('{:,.2f}'.format))

**Observation on above:** There are no huge and obvious differences between the mean scores across these variables for customers who did or did not exit. The widest gaps in means are for Age and Balance, then CreditScore and NumOfProducts. Those who exited are on average older with a larger balance, slightly fewer avg products, and a slightly lower avg CreditScore.

## CreditScore

In [ ]:
# Basic Stats
df[['CreditScore','Exited']].groupby('Exited').describe().apply(lambda s: s.apply('{:,.2f}'.format))

In [ ]:
# Box Plots
plt.title("CreditScore Distributions Comparison", fontsize=14, fontweight='bold')
ax = sns.boxplot(data=df, x='CreditScore', y='Exited', orient='h');
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

In [ ]:
# Violin Plots
plt.title("CreditScore Distributions Comparison", fontsize=14, fontweight='bold')
ax = sns.violinplot(data=df, x='CreditScore', y='Exited', orient='h');
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

In [ ]:
# Histogram
# Control hue_order and palette to display minority class in orange on top: hue_order=1,0 and palette=cp2
plt.figure(figsize=(8,4))
plt.title("CreditScore Distributions Comparison", fontsize=14, fontweight='bold')
ax = sns.histplot(data=df, x='CreditScore', hue='Exited', hue_order=[1,0], palette=cp2, binwidth=25, alpha=0.75);
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

**Interpretation:** The distributions for Exited/Non-Exited CreditScores are nearly identical. Those who exited peak at just a slightly lower CreditScore (625) than those who did not (675).

## Age

In [ ]:
# Histogram
# Control hue_order and palette to display minority class in orange on top: hue_order=1,0 and palette=cp2


In [ ]:
# Violin Plots


In [ ]:
# Box Plots


In [ ]:
# Basic Stats
df[['Age','Exited']].groupby('Exited').describe().apply(lambda s: s.apply('{:,.2f}'.format))

**Interpretation:** 

## Tenure

In [ ]:
# Histogram
# Control hue_order and palette to display minority class in orange on top: hue_order=1,0 and palette=cp2


In [ ]:
# Violin Plots


In [ ]:
# Box Plots


In [ ]:
# Basic Stats


**Interpretation:** 

## Balance

**Interpretation:** 

## EstimatedSalary

In [ ]:
# Histogram
# Control hue_order and palette to display minority class in orange on top: hue_order=1,0 and palette=cp2


In [ ]:
# Violin Plots


In [ ]:
# Box Plots


In [ ]:
# Basic Stats
df[['EstimatedSalary','Exited']].groupby('Exited').describe().apply(lambda s: s.apply('{:,.2f}'.format))

**Interpretation:** 

### Summary of Continuous Features Analysis

#### No Apparent Predictive Relevance
- `Tenure`: tenure has a larger IQR, but otherwise very similar distributions, means, and medians
- `EstimatedSalary`: averages and distributions are nearly identical

#### Perhaps Small Predictive Relevance
- `CreditScore`: **small if any predictive relevance**: there are small distribution differences

#### Likely of Some Relevance
- `Age`: churners average 9 years older than non-churners
- `Balance`: churners have an approximately 15% higher average balance


# Explore Categorical and Ordinal Features
- Geography
- Gender
- NumOfProducts
- HasCrCard
- IsActiveMember
- and Target: Exited

In [ ]:
# Create a variable to filter for categorical and ordinal features

# Preview these features


In [ ]:
# View number of unique categories for each cateogrical and ordinal feature
df[cat].nunique()

## Geography

In [ ]:
# Number of customers per category
df['Geography'].value_counts()

In [ ]:
# Countplot for all customers
plt.title("Geography Distribution for All Customers", fontsize=14, fontweight='bold')
ax = sns.countplot(data=df, x='Geography', color=blue);
ax.set(xlabel = '');

In [ ]:
# Countplot of churners and non-churners for this category
plt.title("Geographical Churn Count Comparisons", fontsize=14, fontweight='bold')
ax = sns.countplot(data=df, x='Geography', hue='Exited');
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

In [ ]:
# Barplot of churn rates for this category
plt.title("Geographical Churn Rates", fontsize=14, fontweight='bold')
ax = sns.barplot(data=df, x='Geography', y='Exited', ci=None, color=blue);
# ax.set(xlabel = 'Custom x axis label', ylabel='Custom y axis label');

In [ ]:
# Calculate Churn Rate for this category
df[['Geography','Exited']].groupby('Geography').mean()

**Interpretation**: German customers are churning at nearly twice the rate of those from France and Spain!

## Gender

In [ ]:
# Count of Customers per Category


In [ ]:
# Count of Customers per Category


In [ ]:
# Countplot of churners and non-churners for this category


In [ ]:
# Barplot of churn rates for this category


In [ ]:
# Calculate Churn Rate for this category


**Interpretation**: Women churn at nearly a 10% higher rate than men: Women at 25%; Men at 16.45%.

## NumOfProducts

In [ ]:
# Count of Customers per Category
df['NumOfProducts'].value_counts()

**Interpretation**: 

## HasCrCard

**Interpretation**: 

## IsActiveMember

**Interpretation**: 

### Summary of Categorical and Ordinal Feature Analysis



# Explore Relationships between Numeric Variables

In [ ]:
df.info()

In [ ]:
# Create variable to hold all features with numeric data types
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html
num = list(df.select_dtypes(include=['number']).columns)
df[num].info()

In [ ]:
# Calculate Pearson Correlations for Numeric Features
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html 
df[num].corr()

In [ ]:
# Heatmap Pearson Correlations for Numeric Features
# https://seaborn.pydata.org/generated/seaborn.heatmap.html
mask = np.triu(np.ones_like(df[num].corr()))
plt.figure(figsize=(7,4))
plt.title("Correlation Heatmap", fontsize=14, fontweight='bold')
ax = sns.heatmap(df[num].corr(), cmap=cpd, vmin=-1, vmax=1, mask=mask, annot=True);

**Interpretation**:
- There are no glaring multicollinearity problems with this data set.

# Data Prep
Prepare all fields for machine learning.

In [ ]:
# Preview current dataframe
df.head()

In [ ]:
# Overview current dataframe fundamentals
df.info()

## Convert Gender to 0 or 1

In [ ]:
# Convert gender to binary integer: Male = 0, Female = 1


## Convert Geography to numeric

In [ ]:
# Convert Geography to integer: France = 0, Spain = 1, Germany = 2


## Write to CSV

Write to CSV file, named `churn_cleaned.csv`

In [ ]:
# Write to CSV
